# Narrowing your query and digging into your results
We previously covered the basics of working with voeventdb.remote in tutorial 1. 

In this notebook, we'll demonstrate the use of filters to narrow down your query, and demonstrate the 'helper classes' you can use to easily access the details for a particular packet.

As before, we'll switch on 'DEBUG' level logging, to see the the HTTP requests go whizzing by.

In [ ]:
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1

We've already briefly looked at the ``stream_count`` endpoint, and mentioned how VOEvents come in three flavours of [role](http://voevent.readthedocs.org/en/latest/reading.html#event-roles), 'observation', 'utility', and 'test'.
Let's remind ourselves what the default stream_count output looks like:

In [ ]:
apiv1.stream_count()

## Using filters

Quite obviously, a number of those streams are 'junk', they contain only test-packets used to verify that the VOEvent infrastructure is up and working correctly. For scientific work, we'll want to filter those out. 

Fortunately, we can ask the voeventdb server to do the filtering work for us. The voeventdb.remote library comes with an easy-to-use list of filters, stored as ``voeventdb.remote.apiv1.FilterKeys``. To see what's available, you can either
use the IPython interface as in the cell below, or simply check the 
[relevant section](http://voeventdbremote.readthedocs.org/en/latest/reference/index.html#voeventdb.remote.apiv1.FilterKeys)
of the documentation. 

Full definitions of the filter-keys (and example filter-values) can be found [in the voeventdb server docs](http://voeventdb.readthedocs.org/en/latest/apiv1/queryfilters.html#apiv1-filters), but we'll cover most of them in this notebook - read on.

In [ ]:
#Alias voeventdb.remote.apiv1.FilterKeys to just 'FilterKeys', for brevity
from voeventdb.remote.apiv1 import FilterKeys

In [ ]:
## To see the list of filters, you can use tab-completion:
# FilterKeys.
## Or the ipython doc-lookup magic, by prefixing with ``??``:
# ??FilterKeys

So: we were trying to filter out the test-packets. ``FilterKeys.role`` sounds promising. To apply a filter, or multiple filters, we simply define a dictionary with the filters we want to apply, and then pass it to the relevant query-function, like this:

In [ ]:
my_filters = { FilterKeys.role: 'observation' }

In [ ]:
apiv1.stream_count(my_filters)

Great! That's a much shorter list, and it only contains scientifically interesting streams. Still, those numbers are pretty large (mainly for *Swift*). It might be useful to get a smaller representative sample. How many packets will we get if we limit our query to a single week?

In [ ]:
from datetime import datetime, timedelta
start_date = datetime(2015,12,1)
my_filters = { 
    FilterKeys.role: 'observation',
    FilterKeys.authored_since: start_date,
    FilterKeys.authored_until: start_date + timedelta(days=7)
    }
my_filters

In [ ]:
apiv1.stream_count(my_filters)

Ok, so there's still a lot of SWIFT packets there. We'll come back to those later, for now let's take a look at the Fermi entries. To do that, we'll grab the 'stream' value from above, and add it to the filter-dictionary:

In [ ]:
my_filters[FilterKeys.stream]='nasa.gsfc.gcn/Fermi'
my_filters

OK, so now if we apply the filters to ``stream_count``, we only get back one entry:

In [ ]:
apiv1.stream_count(filters=my_filters)

Not particularly helpful, but at least everything is working as expected. Now, the neat thing about the voeventdb filters is that they can be applied to **any** query-endpoint - so we can just re-use the filter-dictionary to get back a list of IVORNs:

In [ ]:
fermi_ivorns = apiv1.ivorn(filters=my_filters)
print("Retrieved",len(fermi_ivorns),"IVORNs, as expected")
fermi_ivorns

Now we're getting somewhere! We can clearly see a few subcategories of Fermi packets - alerts, flight positions, ground positions, final positions. We can refine the list even further by filtering on a substring of the IVORN. For example, we might want to filter to just those IVORNs containing flight positions (**note this is case-sensitive!**):

In [ ]:
my_filters[FilterKeys.ivorn_contains] = 'Flt_Pos'
my_filters

In [ ]:
fermi_ivorns = apiv1.ivorn(filters=my_filters)
fermi_ivorns

As in tutorial 1, we can inspect the details of any given packet. This VOEvent is a good example, as it includes details of the event co-ordinates and timestamp, and also references an earlier VOEvent:

In [ ]:
synopsis_dict = apiv1.synopsis(fermi_ivorns[0])
synopsis_dict

## Ready-made 'helper' classes for parsing output
Nested dictionaries can be kind of a pain to work with. If you want, you can use voeventdb.remote's 
[Synopsis](http://voeventdbremote.readthedocs.org/en/latest/reference/index.html#voeventdb.remote.helpers.Synopsis)
'helper' class to parse this into an easy-to use object. This prints with nicer formatting:

In [ ]:
from voeventdb.remote.helpers import Synopsis

In [ ]:
fermi_synopsis = Synopsis(synopsis_dict)
# synopsis
print(fermi_synopsis)

You can easily access the values (with the ever-handy IPython autocompletion):

In [ ]:
fermi_synopsis.author_ivorn

In [ ]:
fermi_synopsis.references

And it parses any 'sky_events' / co-ordinates into 
[astropy.coordinates](http://astropy.readthedocs.org/en/stable/coordinates/index.html) classes, which come with a bunch of features covered in the astropy docs.

In [ ]:
fermi_synopsis.sky_events

In [ ]:
e0 = fermi_synopsis.sky_events[0]

In [ ]:
print(type(e0.position))
e0.position

In [ ]:
print(e0.position.ra.deg)
print(e0.position.ra.hms)

In [ ]:
print(type(e0.position_error))
e0.position_error.deg

## Filtering using a cone-search (AKA spatial queries)

In [ ]:
cone = str([e0.position.ra.deg, e0.position.dec.deg, 1])
cone

In [ ]:
new_filters = {
    FilterKeys.role: 'observation',
    FilterKeys.cone: cone
    }
nearby_ivorns = apiv1.ivorn(new_filters)
nearby_ivorns

In [ ]:
print(Synopsis(apiv1.synopsis(nearby_ivorns[0])))

In [ ]:
## Recall the Fermi-detected event-details, for comparison:
print(fermi_synopsis)

OK so those two events are separated by more than a year, and there are many such *Swift* BAT sub-threshold events scattered all over the sky, so it's likely just a coincidence. 

(Optional exercise - add extra filters to only return events within an interval of one month before / after the Fermi-detected event.)